# Preparation of the environment

## Imports from the Python Standard Library

In [ ]:
import sys

## Installations

In [ ]:
%%capture installation_log
# Miniconda
!wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local
!rm Miniconda3-latest-Linux-x86_64.sh
# RDKit
!conda install -qyc conda-forge rdkit
# DeepSMILES
#23456789|123456789|123456789|123456789|123456789|123456789|123456789|123456789
!wget https://raw.githubusercontent.com/mateuszrezler/deepsmiles/master/\
    deepsmiles/encode.py
# Update system path
sys.path.append('/usr/local/lib/python3.7/site-packages/')